<a href="https://colab.research.google.com/github/amitshmidov/geometric_learning_project/blob/main/geometric_learning_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>